In [ ]:
import requests

class FastChatQuery:
    def __init__(self, model, examples=[]):
        self.model = model
        self.examples = examples

    def query_task(self, text):
        raise NotImplementedError

    def post(self, text):
        url = "http://localhost:8000/v1/chat/completions"
        payload = {
            "model": self.model,
            "messages": self.query_task(text)
        }
        response = requests.post(url, json=payload)
        response = response.json()
        result = response["choices"][0]["message"]["content"]
        return result


class QuestionQuery(FastChatQuery):
    def query_task(self, text):
        messages = []
        # Adding examples
        for example in self.examples:
            messages.append({"role": "user", "content": f"Generate a list of questions that can be answered with information from the following text snippet:\n{example['text']}"})
            messages.append({"role": "assistant", "content": example['questions']})
        # Adding the new text
        messages.append({"role": "user", "content": f"Generate a list of questions that can be answered with information from the following text snippet:\n{text}"})
        return messages


class AnswerQuery(FastChatQuery):
    def query_task(self, text, question):
        messages = []
        # Adding examples
        for example in self.examples:
            messages.append({"role": "user", "content": f"Generate an answer to the following question based on the provided text snippet.\nText: {example['text']}\nQuestion: {example['question']}"})
            messages.append({"role": "assistant", "content": example['answer']})
        # Adding the new text and question
        messages.append({"role": "user", "content": f"Generate an answer to the following question based on the provided text snippet.\nText: {text}\nQuestion: {question}"})
        return messages

    def post(self, text, question):
        url = "http://localhost:8000/v1/chat/completions"
        payload = {
            "model": self.model,
            "messages": self.query_task(text, question)
        }
        response = requests.post(url, json=payload)
        response = response.json()
        result = response["choices"][0]["message"]["content"]
        return result


In [ ]:
QUESTION_EXAMPLES = [
    {
        "text": "Witchfinder General is a 1968 British period horror film directed by Michael Reeves. The screenplay, by Reeves and Tom Baker, was based on Ronald Bassett's novel Witchfinder General. The plot follows Roundhead soldier Richard Marshall, played by Ian Ogilvy, who pursues witchfinder general Matthew Hopkins, played by Vincent Price (pictured), and his assistant John Stearne, played by Robert Russell. The low-budget film was produced by Tigon British Film Productions. In the United States, the film was retitled The Conqueror Worm by its distributor American International Pictures to link it with their earlier series of Edgar Allan Poe adaptations. Witchfinder General became a cult film, partially attributable to Reeves's death nine months after its release. Critics' praise for the film highlighted its direction, performances, and musical score by Paul Ferris. In 2005, the magazine Total Film named Witchfinder General the 15th-greatest horror film of all time.",
        "questions": "Q1: Who directed the film Witchfinder General?\nQ2: What is the novel that the screenplay of Witchfinder General was based on?\nQ3: Who played the character Richard Marshall in the film?\nQ4: What was the film's title in the United States?\nQ5: Why did the film become a cult film?\nQ6: Who composed the musical score for Witchfinder General?\nQ7: What recognition did the film receive from Total Film magazine in 2005?"
    },
    {
        "text": "After the Doolittle Raid in April 1942, in which American planes bombed the Japanese mainland, the Imperial General Headquarters directed Noborito to develop a retaliatory bombing capability against the U.S.[4] In summer 1942, Noborito investigated several proposals, including long-range bombers that could make one-way sorties from Japan to cities on the U.S. West Coast, and small bomb-laden seaplanes which could be launched from submarines.[5] On September 9, 1942, the latter was tested in the Lookout Air Raid, in which a Yokosuka E14Y seaplane was launched from a submarine off the Oregon coast. Warrant Officer Nobuo Fujita dropped two large incendiary bombs in Siskiyou National Forest in the hopes of starting a forest fire and safely returned to the submarine; however, response crews spotted the plane and contained the small blazes. The program was cancelled by the Imperial Navy.[6]",
        "questions": "Q1: What event prompted the Imperial General Headquarters to develop a retaliatory bombing capability against the U.S.?\nQ2: What were some of the proposals investigated by Noborito in summer 1942?\nQ3: What happened during the Lookout Air Raid on September 9, 1942?\nQ4: Who was Nobuo Fujita and what was his role during the Lookout Air Raid?\nQ5: Why was the retaliatory bombing program cancelled by the Imperial Navy?"
    },
]


ANSWER_EXAMPLES = [
    {
        "text": "Witchfinder General is a 1968 British period horror film directed by Michael Reeves. The screenplay, by Reeves and Tom Baker, was based on Ronald Bassett's novel Witchfinder General. The plot follows Roundhead soldier Richard Marshall, played by Ian Ogilvy, who pursues witchfinder general Matthew Hopkins, played by Vincent Price (pictured), and his assistant John Stearne, played by Robert Russell. The low-budget film was produced by Tigon British Film Productions. In the United States, the film was retitled The Conqueror Worm by its distributor American International Pictures to link it with their earlier series of Edgar Allan Poe adaptations. Witchfinder General became a cult film, partially attributable to Reeves's death nine months after its release. Critics' praise for the film highlighted its direction, performances, and musical score by Paul Ferris. In 2005, the magazine Total Film named Witchfinder General the 15th-greatest horror film of all time.",
        "question": "Who directed the film Witchfinder General?",
        "answer": "The film Witchfinder General was directed by Michael Reeves."
    },
    {
        "text": "After the Doolittle Raid in April 1942, in which American planes bombed the Japanese mainland, the Imperial General Headquarters directed Noborito to develop a retaliatory bombing capability against the U.S.[4] In summer 1942, Noborito investigated several proposals, including long-range bombers that could make one-way sorties from Japan to cities on the U.S. West Coast, and small bomb-laden seaplanes which could be launched from submarines.[5] On September 9, 1942, the latter was tested in the Lookout Air Raid, in which a Yokosuka E14Y seaplane was launched from a submarine off the Oregon coast. Warrant Officer Nobuo Fujita dropped two large incendiary bombs in Siskiyou National Forest in the hopes of starting a forest fire and safely returned to the submarine; however, response crews spotted the plane and contained the small blazes. The program was cancelled by the Imperial Navy.[6]",
        "question": "What event prompted the Imperial General Headquarters to develop a retaliatory bombing capability against the U.S.?",
        "answer": "The Doolittle Raid in April 1942, in which American planes bombed the Japanese mainland, prompted the Imperial General Headquarters to develop a retaliatory bombing capability against the U.S."
    },
]


In [ ]:

question_query = QuestionQuery("vicuna-13b", QUESTION_EXAMPLES)
new_text = """The "Fella" meme originated in May 2022 when Twitter artist Kamil Dyszewski (@Kama_Kamilia) started adding modified pictures of a Shiba Inu dog to photographs from Ukraine. The Shiba Inu breed had already gained popularity in online culture through the "doge" meme since 2010. Dyszewski created custom "Fellas" for those who donated to the Georgian Legion. NAFO (NATO and Friends) was established on May 24, 2022, with a significant presence of current and retired service members from Ukrainian and NATO militaries, as well as Eastern Europeans and the Eastern European diaspora. The group, estimated to have "tens of thousands" of associates as of September 2022, incorporates the "Fellas" in edited images, TikTok-style videos of Ukrainian troops with dance music, and war footage to mock Russia's military and praise Ukraine's soldiers. NAFO uses the term "vatniks" to refer to their opponents both online and on the battlefield. Some NAFO members specifically focus on countering the "whataboutism" defense of the Russian invasion."""
questions = question_query.post(new_text)
print(questions)

answer_query = AnswerQuery("vicuna-13b", ANSWER_EXAMPLES)
for question in questions.split('\n'):
    first_word = question.split(' ')[0]
    if first_word.startswith('Q') and first_word.endswith(':'):
        question = question[len(first_word)+1:]
    answer = answer_query.post(new_text, question)
    print(f'{question}\n{answer}')
